In [30]:
import mujoco
import gym
import tianshou as ts
import numpy as np
import torch as tc
import numpy as np
from torch import nn

from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger


In [37]:

if tc.cuda.is_available():
    # gpu_num = input('Enter GPU number: ')
    gpu_num = 0
    if gpu_num == 'all' or gpu_num == '':
        device = tc.device('cuda')
    else:
        device = tc.device(f'cuda:{gpu_num}')
    print('CUDA is available!')
    # SET CUDA_LAUNCH_BLOCKING enable
    # clean cuda memory
    tc.cuda.empty_cache()

    # clear cuda memory
    # tc.cuda.reset_max_memory_allocated()
elif tc.backends.mps.is_available():
    device = tc.device('mps')
    print('MPS is available!')
else:
    device = tc.device('cpu')
    print('CPU...')

MPS is available!


In [2]:
env = gym.make('custom_env:XiaoTian-v0')
obs = env.reset()

/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:64: UserWarning: WARN: A Box action space maximum and minimum values are equal.
  logger.warn("A Box action space maximum and minimum values are equal.")


custom_env:XiaoTian-v0
custom_env:XiaoTian-v0

In [43]:
train_envs = ts.env.DummyVectorEnv(
    [lambda: gym.make('custom_env:XiaoTian-v0') for _ in range(100)])
test_envs = ts.env.DummyVectorEnv(
    [lambda: gym.make('custom_env:XiaoTian-v0') for _ in range(100)])


In [126]:
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n

class BasePolicyNet(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape)),
        )

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, tc.Tensor):
            obs = tc.tensor(obs, dtype=tc.float)
        batch = obs.shape[0]
        logits = self.model(obs.view(batch, -1))
        return logits, state

class saQsaNet(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape) + np.prod(action_shape), 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 1),
        )

    def forward(self, obs, act, state=None, info={}):#obs is observation, state is hidden state
        if not isinstance(obs, tc.Tensor):
            obs = tc.tensor(obs, dtype=tc.float)
        if not isinstance(act, tc.Tensor):
            act = tc.tensor(act, dtype=tc.float)
        batch = obs.shape[0]
        x = tc.cat([obs.view(batch, -1), act.view(batch, -1)], dim=1)
        q_value = self.model(x)
        return q_value

In [114]:

class SACNet(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape)),
        )
        self.model2 = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape)), nn.Sigmoid(),
        )

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, tc.Tensor):
            obs = tc.tensor(obs, dtype=tc.float)
        batch = obs.shape[0]
        logits = (self.model(obs.view(batch, -1)),
                  self.model2(obs.view(batch, -1)))

        return logits, state
SACNet


DDPG parameters
Parameters
actor (torch.nn.Module) – the actor network following the rules in BasePolicy. (s -> logits)

actor_optim (torch.optim.Optimizer) – the optimizer for actor network.

critic (torch.nn.Module) – the critic network. (s, a -> Q(s, a))

critic_optim (torch.optim.Optimizer) – the optimizer for critic network.

tau (float) – param for soft update of the target network. Default to 0.005.

gamma (float) – discount factor, in [0, 1]. Default to 0.99.

exploration_noise (BaseNoise) – the exploration noise, add to the action. Default to GaussianNoise(sigma=0.1).

reward_normalization (bool) – normalize the reward to Normal(0, 1), Default to False.

estimation_step (int) – the number of steps to look ahead. Default to 1.

action_scaling (bool) – whether to map actions from range [-1, 1] to range [action_spaces.low, action_spaces.high]. Default to True.

action_bound_method (str) – method to bound action to range [-1, 1], can be either “clip” (for simply clipping the action) or empty string for no bounding. Default to “clip”.

action_space (Optional[gym.Space]) – env’s action space, mandatory if you want to use option “action_scaling” or “action_bound_method”. Default to None.

lr_scheduler – a learning rate scheduler that adjusts the learning rate in optimizer in each policy.update(). Default to None (no lr_scheduler).

In [127]:
DDPGaNet = BasePolicyNet(state_shape, action_shape)
DDPGcNet = saQsaNet(state_shape, action_shape)
DDPGAoptim = tc.optim.Adam(DDPGaNet.parameters(), lr=1e-3)
DDPGCoptim = tc.optim.Adam(DDPGcNet.parameters(), lr=1e-3)
DDPGpolicy = ts.policy.DDPGPolicy(actor=DDPGaNet, actor_optim=DDPGAoptim, critic=DDPGcNet, critic_optim=DDPGCoptim, estimation_step=3)
DDPGtrain_collector = ts.data.Collector(
    DDPGpolicy, train_envs, ts.data.VectorReplayBuffer(20000, 100), exploration_noise=True)
DDPGtest_collector = ts.data.Collector(
    DDPGpolicy, test_envs, exploration_noise=True)


SAVPolicy
Parameters
actor (torch.nn.Module) – the actor network following the rules in BasePolicy. (s -> logits)

actor_optim (torch.optim.Optimizer) – the optimizer for actor network.

critic1 (torch.nn.Module) – the first critic network. (s, a -> Q(s, a))

critic1_optim (torch.optim.Optimizer) – the optimizer for the first critic network.

critic2 (torch.nn.Module) – the second critic network. (s, a -> Q(s, a))

critic2_optim (torch.optim.Optimizer) – the optimizer for the second critic network.

tau (float) – param for soft update of the target network. Default to 0.005.

gamma (float) – discount factor, in [0, 1]. Default to 0.99.

torch.Tensor, torch.optim.Optimizer) or float alpha ((float,) – entropy regularization coefficient. Default to 0.2. If a tuple (target_entropy, log_alpha, alpha_optim) is provided, then alpha is automatically tuned.

reward_normalization (bool) – normalize the reward to Normal(0, 1). Default to False.

exploration_noise (BaseNoise) – add a noise to action for exploration. Default to None. This is useful when solving hard-exploration problem.

deterministic_eval (bool) – whether to use deterministic action (mean of Gaussian policy) instead of stochastic action sampled by the policy. Default to True.

action_scaling (bool) – whether to map actions from range [-1, 1] to range [action_spaces.low, action_spaces.high]. Default to True.

action_bound_method (str) – method to bound action to range [-1, 1], can be either “clip” (for simply clipping the action) or empty string for no bounding. Default to “clip”.

action_space (Optional[gym.Space]) – env’s action space, mandatory if you want to use option “action_scaling” or “action_bound_method”. Default to None.

lr_scheduler – a learning rate scheduler that adjusts the learning rate in optimizer in each policy.update(). Default to None (no lr_scheduler).

In [115]:
SACaNet = SACNet(state_shape, action_shape)
SACc1Net = saQsaNet(state_shape, action_shape)
SACc2Net = saQsaNet(state_shape, action_shape)
SACaOptim = tc.optim.Adam(SACaNet.parameters(), lr=1e-3)
SACc1Optim = tc.optim.Adam(SACc1Net.parameters(), lr=1e-3)
SACc2Optim = tc.optim.Adam(SACc2Net.parameters(), lr=1e-3)
SACppolicy = ts.policy.SACPolicy(actor=SACaNet, actor_optim=SACaOptim, critic1=SACc1Net,
                                 critic1_optim=SACc1Optim, critic2=SACc2Net, critic2_optim=SACc2Optim, estimation_step=3)


SACtrain_collector = ts.data.Collector(
    SACppolicy, train_envs, ts.data.VectorReplayBuffer(20000, 100), exploration_noise=True)
SACtest_collector = ts.data.Collector(
    SACppolicy, test_envs, exploration_noise=True)


In [ ]:
DDPGwriter = SummaryWriter('log/DDPGdqn')
DDPGlogger = TensorboardLogger(DDPGwriter)
result = ts.trainer.offpolicy_trainer(
    DDPGpolicy, DDPGtrain_collector, DDPGtest_collector,
    max_epoch=2000, step_per_epoch=100, step_per_collect=10,
    update_per_step=0.1, episode_per_test=15, batch_size=64,
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    logger = DDPGlogger)
print(f'Finished training! Use {result["duration"]}')
tc.save(DDPGpolicy.state_dict(), 'tianshou_results/log/DDPGdqn.pth')
tc.load(DDPGpolicy.state_dict(), 'tianshou_results/log/DDPGdqn.pth')

Epoch #1:   0%|          | 0/100 [00:00<?, ?it/s]/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/tianshou/data/collector.py:255: UserWarning: n_step=10 is not a multiple of #env (100), which may cause extra transitions collected into the buffer.
  warnings.warn(
Epoch #1: 101it [00:00, 988.38it/s, env_step=100, len=1, loss/actor=-1.434, loss/critic=4.410, n/ep=100, n/st=100, rew=0.45]                         


Epoch #1: test_reward: -0.002026 ± 0.206586, best_reward: 0.460380 ± 0.196909 in #0


Epoch #2: 101it [00:00, 954.51it/s, env_step=200, len=1, loss/actor=-1.933, loss/critic=3.947, n/ep=100, n/st=100, rew=-0.00]                         


Epoch #2: test_reward: 0.026730 ± 0.182932, best_reward: 0.460380 ± 0.196909 in #0


Epoch #3: 101it [00:00, 976.20it/s, env_step=300, len=1, loss/actor=-2.184, loss/critic=3.925, n/ep=100, n/st=100, rew=0.06]                         


...

Epoch #199: 101it [00:00, 1004.11it/s, env_step=19900, len=1, loss/actor=-0.886, loss/critic=0.055, n/ep=100, n/st=100, rew=0.62]                         


Epoch #199: test_reward: 0.634764 ± 0.186261, best_reward: 0.797995 ± 0.153606 in #180


Epoch #200: 101it [00:00, 972.37it/s, env_step=20000, len=1, loss/actor=-0.887, loss/critic=0.052, n/ep=100, n/st=100, rew=0.60]                         

Epoch #200: test_reward: 0.635114 ± 0.187312, best_reward: 0.797995 ± 0.153606 in #180
Finished training! Use 26.10s


AttributeError: 'collections.OrderedDict' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [117]:
writer = SummaryWriter('log/SAC')
logger = TensorboardLogger(writer)
result = ts.trainer.offpolicy_trainer(
    SACppolicy, SACtrain_collector, SACtest_collector,
    max_epoch=200, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=15, batch_size=64,
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    logger=logger)
print(f'Finished training! Use {result["duration"]}')
tc.save(SACppolicy.state_dict(), 'tianshou_results/log/SACdqn.pth')
tc.load(SACppolicy.state_dict(), 'tianshou_results/log/SACdqn.pth')

Epoch #1:   0%|          | 0/10000 [00:00<?, ?it/s]/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/tianshou/data/collector.py:255: UserWarning: n_step=10 is not a multiple of #env (100), which may cause extra transitions collected into the buffer.
  warnings.warn(
Epoch #1: 10001it [00:12, 790.81it/s, env_step=10000, len=1, loss/actor=-2.733, loss/critic1=6.084, loss/critic2=6.084, n/ep=100, n/st=100, rew=-2.33]                           


Epoch #1: test_reward: 1.122193 ± 0.138248, best_reward: 1.122193 ± 0.138248 in #1


Epoch #2: 10001it [00:12, 801.66it/s, env_step=20000, len=1, loss/actor=-2.733, loss/critic1=6.062, loss/critic2=6.062, n/ep=100, n/st=100, rew=-2.36]                           


Epoch #2: test_reward: 1.129613 ± 0.084685, best_reward: 1.129613 ± 0.084685 in #2


Epoch #3: 10001it [00:12, 786.52it/s, env_step=30000, len=1, loss/actor=-2.734, loss/critic1=6.172, loss/critic2=6.172, n/ep=100, n/st=100, rew=-2.36]                           


...

Epoch #199: 10001it [00:12, 785.58it/s, env_step=1990000, len=1, loss/actor=-2.736, loss/critic1=6.112, loss/critic2=6.112, n/ep=100, n/st=100, rew=-2.41]                           


Epoch #199: test_reward: 1.058799 ± 0.089497, best_reward: 1.154821 ± 0.125385 in #163


Epoch #200: 10001it [00:12, 797.98it/s, env_step=2000000, len=1, loss/actor=-2.731, loss/critic1=6.191, loss/critic2=6.191, n/ep=100, n/st=100, rew=-2.45]                           


Epoch #200: test_reward: 1.045275 ± 0.176816, best_reward: 1.154821 ± 0.125385 in #163
Finished training! Use 3139.84s


AttributeError: 'collections.OrderedDict' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [118]:
writer.flush()

In [119]:
tensorboard --logdir=runs

SyntaxError: cannot assign to operator (3224537314.py, line 1)

In [ ]:
policy.eval()
# policy.set_eps(0.05)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/tianshou/data/collector.py:68: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")
/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/gym/envs/mujoco/mujoco_env.py:363: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("XiaoTian-v0", render_mode="rgb_array")
  gym.logger.warn(


{'n/ep': 1,
 'n/st': 1,
 'rews': array([0.88650187]),
 'lens': array([1]),
 'idxs': array([0]),
 'rew': 0.8865018660339451,
 'len': 1.0,
 'rew_std': 0.0,
 'len_std': 0.0}